# Imports

In [1]:
from ibm_watson_machine_learning import APIClient

# 

# Credenciales

In [2]:
#Credenciales de acceso al deploy

wml_credentials = {
    "apikey":"API_KEY_VALUE",
    "url":"https://us-south.ml.cloud.ibm.com"
}


# 

# Seteamos el cliente y el espacio al que deployamos

In [ ]:
#creamos el cliente de ML

wml_client = APIClient(wml_credentials)

wml_client.spaces.list()


In [4]:
#Guardamos el ID del espacio donde se desplegará el modelo y lo seteamos como default

SPACE_ID = "SPACE_ID"

wml_client.set.default_space(SPACE_ID)

'SUCCESS'

# 

# Funcion a  deployar

In [5]:
def deployable_callable():
    """
    Función de python deployable que entrena un modelo para un usuario determinado
    """
    
    #Primero hacemos imports y seteamos configuracion inicial
    
    import ibm_boto3
    from botocore.client import Config
    import requests
    
    
    api_key = 'API_KEY_VALUE'

    project_id = '821deb35-d3d9-4a8d-a5e7-b1823688b22f'

    job_id = '2fdeecb0-524f-4f9d-8a47-c3c9c1df3bb4'
        
    
    
    
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################
    
    def get_bearer_access_token(api_key):
        """
        Funcion que utilizando la api key del usuario, obtiene el bearer token que permite ejecutar un job
        """

        headers = {'Content-Type': 'application/x-www-form-urlencoded',}

        data = 'grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey='+api_key

        response = requests.post('https://iam.cloud.ibm.com/identity/token', headers=headers, data=data)

        access_token = response.json()['access_token']

        return access_token
        
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################    
    
    def execute_job(access_token, project_id, job_id):
        """
        Funcion que recibe un access_token, un job_id y un project_id, y ejecuta el job recibido como parámetro
        """
        headers = {
            'accept': 'application/json',
            'Authorization': 'Bearer '+access_token,
        }

        params = {
            'project_id': project_id,
            'userfs': 'false',
        }

        json_data = {
            'job_run': {
                'name': 'Name',
                'description': 'Description',
                'configuration': {
                    'env_variables': [
                        'key1=value1',
                        'key2=value2',
                    ],
                },
                'job_parameters': [
                    {
                        'name': 'job_param_1',
                        'value': 'The value could be of any type',
                    },
                ],
                'parameter_sets': [
                    {
                        'name': 'paramset_1',
                        'value_set': 'red',
                        'ref': '31abfd90-5497-4605-8ac1-d85896fe6c6f',
                    },
                ],
            },
        }



        run_job_api_url = 'https://api.dataplatform.cloud.ibm.com/v2/jobs/'+job_id+'/runs'

        response = requests.post(run_job_api_url, params=params, headers=headers, json=json_data)

        return response.status_code
        
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################           
    
    def guardar_archivo(user_to_train):
        """
        Funcion que guarda el usuario a entrenar en un txt para que luego el modelo lo levante
        """
    
        bucket = 'pruebasgian-donotdelete-pr-3aeebt0ewfpjpi'

        cos = ibm_boto3.client(service_name='s3',
                                  ibm_api_key_id='IBM_API_KEY_ID',
                                  ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
                                  config=Config(signature_version='oauth'),
                                  endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

        filename = "user_to_train.txt"

        with open(filename, 'wb+') as z:

            response = cos.put_object(Body=user_to_train,
                                      Bucket=bucket,
                                      Key=filename
                                     )

        return response['ResponseMetadata']['HTTPStatusCode']
        
    ##############################################################################################################################
    ##############################################################################################################################
    ##############################################################################################################################      
    

    def score(payload):
        """
        Train and create model method.
        
        """
        
               
        try:
        
            user_to_train = payload['input_data'][0]['values']


            status_guardado_archivo = guardar_archivo(user_to_train)
    
            if status_guardado_archivo != 200:
            
                return {'predictions': [{'values': "Error when saving the text name file to train the model for "+user_to_train+". Please check the logs and try in a few moments!"}]}
            
                
            
            access_token = get_bearer_access_token(api_key)
            
            status_code_execution = execute_job(access_token, project_id, job_id)
            
            if status_code_execution==201:
                return {'predictions': [{'values': "Training Model for "+user_to_train+"!"}]}
            
            return {'predictions': [{'values': "Error when trying to train the model for "+user_to_train+". Please check the logs and try in a few moments!"}]}
        
        
        except Exception as e:
            return {'predictions': [{'values': "Error when trying to train the model for "+user_to_train+". Please check the logs and try in a few moments!"}]}
        
    return score

In [6]:
#Test de la función creada para deployar

function_test = deployable_callable()({
    "input_data": [{
        'values': "gian.grillo"
    }]
})

print(function_test["predictions"][0]["values"])

Training Model for gian.grillo!


# 

# Configuramos el entorno al que deployamos

In [7]:
base_sw_spec_uid = wml_client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")

In [8]:
meta_props = {
    wml_client.repository.FunctionMetaNames.NAME: "Train model function",
    wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: base_sw_spec_uid
}

function_details = wml_client.repository.store_function(meta_props=meta_props, function=deployable_callable)
function_uid = wml_client.repository.get_function_uid(function_details)

In [9]:
wml_client.repository.get_details(function_uid)

{'entity': {'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'python'},
 'metadata': {'created_at': '2022-10-15T22:02:17.456Z',
  'id': '6d2ede50-785f-4bb1-9357-9dcea4226a7d',
  'modified_at': '2022-10-15T22:02:19.848Z',
  'name': 'Train model function',
  'owner': 'IBMid-6650007T1K',
  'space_id': '8a50bf52-17ff-401d-a128-7502ce8436f4'},
 'system': {'warnings': []}}

In [10]:
wml_client.repository.list_functions()

------------------------------------  --------------------  ------------------------  ------
GUID                                  NAME                  CREATED                   TYPE
6d2ede50-785f-4bb1-9357-9dcea4226a7d  Train model function  2022-10-15T22:02:17.002Z  python
fb5b9ce5-55ca-4392-b21c-d81a49847ab6  Train model function  2022-10-15T22:00:25.002Z  python
6d28a0c3-af01-45db-963f-f73e31e54527  Train model function  2022-10-15T21:34:22.002Z  python
------------------------------------  --------------------  ------------------------  ------


# 

# Hacemos el Deploy

In [11]:
metadata = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "Deployment of Train model function",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

function_deployment = wml_client.deployments.create(function_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: '6d2ede50-785f-4bb1-9357-9dcea4226a7d' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b46d225b-7b39-42fb-a219-e6bb74c7464f'
------------------------------------------------------------------------------------------------




In [12]:
wml_client.deployments.list()

------------------------------------  ----------------------------------  -----  ------------------------
GUID                                  NAME                                STATE  CREATED
b46d225b-7b39-42fb-a219-e6bb74c7464f  Deployment of Train model function  ready  2022-10-15T22:02:23.437Z
------------------------------------  ----------------------------------  -----  ------------------------


# 

# Probamos la función deployada

In [13]:
deployment_id = wml_client.deployments.get_uid(function_deployment)


In [14]:
scoring_payload = {
    "input_data": [{
        'values': "gian.grillo"
    }]
}

predictions = wml_client.deployments.score(deployment_id, scoring_payload)


In [16]:
predictions

{'predictions': [{'values': 'Training Model for gian.grillo!'}]}